# Cargar Datos

**Proyecto:** Pipeline MLOps - Predicción de Pago a Tiempo de Créditos

**Autor:** Alexis Jacquet

**Fecha:** 5 de febrero de 2026

---

## Objetivo

Este notebook tiene como objetivo cargar los datos desde un archivo CSV y realizar una primera revisión de la estructura y calidad de los datos. En un entorno productivo, estos datos provendrían de un Data Warehouse o Data Lake de la empresa.

## Dataset

El dataset contiene información sobre créditos financieros y características de los clientes, con el objetivo de predecir si el cliente pagará a tiempo (`Pago_atiempo`).

## 1. Importar Librerías

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Para manejo de rutas
import os
from pathlib import Path

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Semilla para reproducibilidad
np.random.seed(42)

print("✓ Librerías importadas exitosamente")

## 2. Definir Rutas de Archivos

In [ ]:
# Obtener el directorio raíz del proyecto
# Estamos en mlops_pipeline/src/, necesitamos subir 2 niveles
RUTA_RAIZ = Path.cwd().parent.parent
RUTA_DATOS = RUTA_RAIZ / 'Base_de_datos.csv'

print(f"Directorio raíz del proyecto: {RUTA_RAIZ}")
print(f"Ruta del archivo de datos: {RUTA_DATOS}")
print(f"¿El archivo existe?: {RUTA_DATOS.exists()}")

## 3. Cargar el Dataset

In [ ]:
# Cargar datos desde CSV
try:
    df = pd.read_csv(RUTA_DATOS)
    print("✓ Datos cargados exitosamente")
    print(f"\nDimensiones del dataset: {df.shape[0]} filas x {df.shape[1]} columnas")
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo {RUTA_DATOS}")
    print("Por favor, verifica que el archivo existe en la raíz del proyecto.")
except Exception as e:
    print(f"❌ Error al cargar los datos: {str(e)}")

## 4. Exploración Inicial de los Datos

Realizamos una primera inspección visual de los datos para familiarizarnos con su estructura.

In [ ]:
# Mostrar las primeras filas
print("="*80)
print("PRIMERAS 5 FILAS DEL DATASET")
print("="*80)
df.head()

In [ ]:
# Información general del dataset
print("="*80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*80)
df.info()

In [ ]:
# Nombres de las columnas
print("="*80)
print("COLUMNAS DEL DATASET")
print("="*80)
print(f"\nTotal de columnas: {len(df.columns)}\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

In [ ]:
# Tipos de datos por columna
print("="*80)
print("TIPOS DE DATOS POR COLUMNA")
print("="*80)
tipos_datos = df.dtypes.value_counts()
print(f"\nResumen:")
print(tipos_datos)
print(f"\nDetalle por columna:")
print(df.dtypes)

## 5. Valores Nulos

Identificamos la presencia de valores nulos en cada columna.

In [ ]:
# Análisis de valores nulos
print("="*80)
print("ANÁLISIS DE VALORES NULOS")
print("="*80)

nulos = df.isnull().sum()
nulos_pct = (df.isnull().sum() / len(df)) * 100

resumen_nulos = pd.DataFrame({
    'Valores_Nulos': nulos,
    'Porcentaje': nulos_pct
}).sort_values(by='Valores_Nulos', ascending=False)

# Filtrar solo columnas con nulos
resumen_nulos_filtrado = resumen_nulos[resumen_nulos['Valores_Nulos'] > 0]

if len(resumen_nulos_filtrado) > 0:
    print(f"\nColumnas con valores nulos: {len(resumen_nulos_filtrado)}\n")
    print(resumen_nulos_filtrado)
else:
    print("\n✓ No se encontraron valores nulos en ninguna columna")

print(f"\nTotal de valores nulos en todo el dataset: {df.isnull().sum().sum()}")

## 6. Estadísticas Descriptivas Básicas

In [ ]:
# Estadísticas para variables numéricas
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("="*80)
df.describe()

In [ ]:
# Estadísticas para variables categóricas
print("="*80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS")
print("="*80)
df.describe(include='object')

## 7. Análisis de la Variable Objetivo

La variable objetivo es `Pago_atiempo`, que indica si el cliente pagó a tiempo (1) o no (0).

In [ ]:
# Distribución de la variable objetivo
print("="*80)
print("ANÁLISIS DE LA VARIABLE OBJETIVO: Pago_atiempo")
print("="*80)

distribucion_objetivo = df['Pago_atiempo'].value_counts().sort_index()
distribucion_pct = df['Pago_atiempo'].value_counts(normalize=True).sort_index() * 100

resumen_objetivo = pd.DataFrame({
    'Frecuencia': distribucion_objetivo,
    'Porcentaje': distribucion_pct
})

print("\nDistribución:")
print(resumen_objetivo)

# Visualización
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico de barras
ax[0].bar(['No pagó a tiempo (0)', 'Pagó a tiempo (1)'], distribucion_objetivo.values, 
          color=['#e74c3c', '#2ecc71'], alpha=0.7, edgecolor='black')
ax[0].set_title('Distribución de Pago a Tiempo', fontsize=12, fontweight='bold')
ax[0].set_ylabel('Frecuencia')
ax[0].grid(axis='y', alpha=0.3)

# Gráfico de torta
ax[1].pie(distribucion_objetivo.values, labels=['No pagó a tiempo', 'Pagó a tiempo'],
          autopct='%1.1f%%', colors=['#e74c3c', '#2ecc71'], startangle=90)
ax[1].set_title('Proporción de Pago a Tiempo', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Verificar balance de clases
ratio = distribucion_objetivo.min() / distribucion_objetivo.max()
print(f"\nRatio de balance de clases: {ratio:.2f}")
if ratio < 0.5:
    print("⚠️  Advertencia: El dataset presenta desbalance de clases significativo.")
    print("    Se recomienda aplicar técnicas de balanceo durante el entrenamiento del modelo.")
else:
    print("✓ Las clases están razonablemente balanceadas.")

## 8. Resumen de la Carga de Datos

In [ ]:
print("="*80)
print("RESUMEN DE LA CARGA DE DATOS")
print("="*80)
print(f"\n✓ Dataset cargado exitosamente")
print(f"\n📊 Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
print(f"\n📋 Tipos de datos:")
for dtype, count in df.dtypes.value_counts().items():
    print(f"   - {dtype}: {count} columnas")
print(f"\n❓ Valores nulos: {df.isnull().sum().sum():,} ({(df.isnull().sum().sum() / df.size * 100):.2f}% del total)")
print(f"\n🎯 Variable objetivo: Pago_atiempo")
print(f"   - Clase 0 (No pagó a tiempo): {distribucion_objetivo[0]:,} ({distribucion_pct[0]:.1f}%)")
print(f"   - Clase 1 (Pagó a tiempo): {distribucion_objetivo[1]:,} ({distribucion_pct[1]:.1f}%)")
print(f"\n✓ Datos listos para el análisis exploratorio (EDA)")
print("\n" + "="*80)

## 9. Guardar Referencia al DataFrame

El DataFrame `df` está disponible para su uso en el siguiente notebook de análisis exploratorio.

In [ ]:
# Verificar que el DataFrame está cargado en memoria
print(f"DataFrame 'df' disponible en memoria con {df.shape[0]:,} registros")
print("\n✓ Listo para proceder con comprension_eda.ipynb")